# Process OMI HDF-format files to visualize SO2 mass in the atmosphere in a volcanic region

**This notebook is a stopgap measure. The former processing script failed; as it uses IDL we decided not to attempt to fix it, but to develop a basic python alternative. Satellite-based atmospheric gas analysis will fully switch to use Tropomi http://www.tropomi.eu/ once some issues with that processing have been fixed.**

OMI data are provided by NASA and are sent to hatepe.gns.cri.nz (aliased to volcano.gns.cri.nz) daily. A shell script on volcano moves the files to /home/volcano/data/omi/archive.

`omi
└── archive
    ├── 2020-04-01
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t1653-o83566_v003-2020m0401t114126.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t1832-o83567_v003-2020m0401t114119.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2011-o83568_v003-2020m0401t114113.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2150-o83569_v003-2020m0401t114112.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2328-o83570_v003-2020m0401t184121.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t0107-o83571_v003-2020m0401t184102.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t0246-o83572_v003-2020m0401t185719.he5
    │   └── OMI-Aura_L2-OMSO2_2020m0401t0425-o83573_v003-2020m0401t185732.he5
    ├── 2020-04-02
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t1736-o83581_v003-2020m0402t120605.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t1915-o83582_v003-2020m0402t120601.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t2054-o83583_v003-2020m0402t120552.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t2233-o83584_v003-2020m0402t120550.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0012-o83585_v003-2020m0402t143344.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0151-o83586_v003-2020m0402t143855.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0330-o83587_v003-2020m0402t143855.he5
    │   └── OMI-Aura_L2-OMSO2_2020m0402t0508-o83588_v003-2020m0402t203226.he5
    ├── 2020-04-03`
    
Data files have two date/times in their names:
- 1st = acquisition
- 2nd = when processed

The files are 'filed' based on the processing date. Sometimes the same orbit data is processed twice, and we recieve both processed files, in that case the most recent one should be used.

The satellite passes overhead the Equator at approximately 1:45 pm, local time (I assume this is ignoring Daylight Saving, but aren't 100%). This can be used to determine which file, each of which contains a single data swath, is the right one for a particular area. For example, the file suitable for PNG-Solomons is acquired at about 0245 UTC.

##Data for a particular UTC date can be found in the folder named by that date (typically up to about 0500 UTC), and in the folder for the following (typically from about 1600 UTC).

Data are visualized for the following regions (this is slightly less than the original IDL-coded product):
- Papua New Guinea - Solomon Island, coding abbreviation = pngsol
- Vanuatu, coding abbreviation = vanuatu
- Samoa - Fiji - Tonga, coding abbreviation = sft
- New Zealand (North Island only), coding abbreviation = nz
- Kermadec Islands (centred around Raoul Is and Curtis Is), coding abbreviation = kerm

In [ ]:
import os
import glob
import sys

import pandas as pd

import datetime
from datetime import timedelta
import pytz

import numpy as np
import h5py

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

import cartopy.crs as ccrs
from PIL import Image

In [ ]:
def passtime(area, date):
    #find the theoretical pass time for the OMI satellite at the longitude of the area provided
    #satellite nominally passes at 1:45 pm
    
    #timezones
    zones = {'pngsol':'Pacific/Bougainville', 'vanuatu':'Pacific/Bougainville', 'sft':'Pacific/Tongatapu', 'nz':'NZ', 'kerm':'NZ' }
    
    obstime = '13:45:00' # 1:45 pm local time
    dt = date+' '+obstime
    dt = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    
    timezone = pytz.timezone(zones[area])
    dt = timezone.localize(dt)
    dtutc = dt.astimezone(pytz.utc)
    return (dtutc)

In [ ]:
def nearest_file(files, passutc):
    #find nearest files to satellite pass time
    diffmin = 1e9
    nearones = []
    
    for file in files:
        f = os.path.basename(file)
        path = os.path.dirname(file)
        swathdate = f[18:22]+'-'+f[23:25]+'-'+f[25:27]
        swathtime = f[28:30]+':'+f[30:32]
        swathdt = swathdate+' '+swathtime
        swathdt = datetime.datetime.strptime(swathdt, '%Y-%m-%d %H:%M')
        timezone = pytz.timezone('UTC')
        swathdt = timezone.localize(swathdt)
        tdiff = np.abs((swathdt - passutc).total_seconds())
        if tdiff < diffmin:
            diffmin = tdiff
            filemin = f
            pathmin = path
            near = os.path.join(path, f)
            nearones.insert(0,near)
    return (nearones)

In [ ]:
def get_volcdata(volclist, volcname):
    #find appropriate dictionary from list of dictionaries
    for v in volclist:
        if v['name'] == volcname:
            return (v)

In [ ]:
def findfiles(searchdays):
    #list files to check, in the date folders specified
    files = []
    for d in searchdays:
        dir = os.path.join(datadir, d)
        try:
            dirfiles = [os.path.join(dir, file) for file in os.listdir(dir)]
            for file in dirfiles:
                files.append(file)
        except:
            pass
    return (files)

In [ ]:
def visualize(day, f, volclist, volcname, vizdir):
    
    volcdata = get_volcdata(volclist, volcname)
    
    fullname = volcdata['fullname']
    plot_extent = volcdata['plot_extent']
    title_x = volcdata['title_x']
    title_y = volcdata['title_y']
    imlat = volcdata['imlat']
    imlon = volcdata['imlon']
    imlon -= 180
    xlocs = volcdata['xlocs']
    ylocs = volcdata['ylocs']
    
    #image swath date-time
    imf = os.path.basename(f)
    swathdate = imf[18:22]+'-'+imf[23:25]+'-'+imf[25:27]
    swathtime = imf[28:30]+':'+imf[30:32]
    swathdt = swathdate+' '+swathtime

    file = h5py.File(f, 'r')

    dataFields=file['HDFEOS']['SWATHS']['OMI Total Column Amount SO2']['Data Fields']
    geolocation=file['HDFEOS']['SWATHS']['OMI Total Column Amount SO2']['Geolocation Fields']
    SDS_NAME='ColumnAmountSO2_TRL'
    data=dataFields[SDS_NAME]

    valid_min=data.attrs['ValidRange'][0]
    valid_max=data.attrs['ValidRange'][1]
    map_label=data.attrs['Units'].decode()

    #get necessary attributes 
    fv=data.attrs['_FillValue']
    mv=data.attrs['MissingValue']
    try:
        offset=data.attrs['Offset']
    except:
        offset=np.array([0],dtype='float')
    try:
        scale=data.attrs['ScaleFactor']
    except:
        scale=np.array([1],dtype='float')

    #get lat and lon information 
    lat=geolocation['Latitude'][:]
    limit = -90
    b = np.ma.MaskedArray(lat, lat<limit)
    min_lat=np.min(b)
    max_lat=np.max(b)
    lon=geolocation['Longitude'][:]
    limit = -180
    b = np.ma.MaskedArray(lon, lon<limit)
    min_lon=np.min(b)
    max_lon=np.max(b)

    #get the data as an array and mask fill/missing values
    dataArray=data[:]
    dataArray[dataArray==fv]=np.nan
    dataArray[dataArray==mv]=np.nan
    dataArray = scale * (dataArray - offset)

    #get statistics about data
    average=np.nanmean(dataArray)
    stdev=np.nanstd(dataArray)
    median=np.nanmedian(dataArray)

    dataArray = np.ma.masked_array(dataArray, np.isnan(dataArray))

    #plot
    fig = plt.figure(figsize=(15,15))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    ax.set_extent(plot_extent)
    ax.coastlines(resolution='10m')
    ax.gridlines(draw_labels=True, xlocs=xlocs, ylocs=ylocs)

    title_text = 'SO2 (3 km plume centre) - '+fullname+' for '+day+'\nshowing data at '+swathdt+' UTC'

    ax.text(title_x, title_y, title_text, transform=ccrs.PlateCarree(), horizontalalignment='left', fontsize=14)

    #GNS logo
    img = Image.open('GNS_logo.png')
    imagebox = OffsetImage(img, zoom=0.075)
    imagebox.image.axes = ax
    ab = AnnotationBbox(imagebox, [imlon, imlat], pad=0, frameon=False)
    ax.add_artist(ab)
    
    #missing data, when data array contains the missing or fill value (the same value)
    missing = data[data[:]==mv]
    lat_missing = lat[data[:]==mv]
    lon_missing = lon[data[:]==mv]
    ax.plot(lon_missing, lat_missing, marker='.', color='red', markersize=8, alpha=0.05, transform=ccrs.PlateCarree())

    #contour data
    cs = plt.contourf(lon, lat, dataArray, levels=[0, 0.8, 1.5, 2.2, 3.0], colors=['white', 'purple', 'green', 'yellow', 'red'],
                     extend='max', alpha=0.5, transform=ccrs.PlateCarree())
    fig.colorbar(cs, ax=ax, shrink=0.5, label='Total column SO2 (DU)')
    
    #tight layout does not work with cartopy
#     fig.canvas.draw()
#     fig.tight_layout()

    #save plot
    plt.gcf()
    savefile = os.path.join(vizdir,day+'_'+volcname+'.png')
    plt.savefig(savefile)
    #tight layout does not work with cartopy
    os.system('convert %s -trim +repage %s' %(savefile,savefile))

    file.close()

## Setup

In [ ]:
#data for volcanoes to visualize
volcanoes = [{ 'name': 'vanuatu', 'fullname': 'Vanuatu', 'plot_extent': [161, 179, -22, -8], 'title_x': 161.5, 'title_y': -9,
             'imlat': -20.75, 'imlon': 178.25, 'xlocs': [158,162,166,170,174,178,182], 'ylocs': [-6,-10,-14,-18,-22]},
             { 'name': 'nz', 'fullname': 'New Zealand', 'plot_extent': [170, 180, -42, -33], 'title_x': 170.25, 'title_y': -33.5,
             'imlat': -41.25, 'imlon': 179.5, 'xlocs': [168,172,176,180], 'ylocs': [-42,-40,-36,-32]},             
             { 'name': 'sft', 'fullname': 'Samoa - Fiji - Tonga', 'plot_extent': [172, 192, -24, -10], 'title_x': 172.5, 'title_y': -11,
             'imlat': -23, 'imlon': 191, 'xlocs': [172,176,180,-180,-176,-172,-168], 'ylocs': [-24,-20,-16,-12,-10]},
             { 'name': 'kerm', 'fullname': 'Kermadec Islands', 'plot_extent': [172, 188, -33, -21], 'title_x': 172.5, 'title_y': -22,
             'imlat': -32, 'imlon': 187, 'xlocs': [172,176,180,-180,-176,-172], 'ylocs': [-33,-32,-26,-22,-21]},
             { 'name': 'pngsol', 'fullname': 'Papua New Guinea - Solomon Islands', 'plot_extent': [142, 164, -14, 0], 'title_x': 142.5, 'title_y': -1,
             'imlat': -12.5, 'imlon': 163, 'xlocs': [142,144,148,152,156,160,164], 'ylocs': [-14,-10,-6,-2,0]},
            ]

#folder containing data archive
# datadir = '/home/sherburn/Work/GeoNet/omi/archive'
datadir = '/run/user/4007/gvfs/sftp:host=volcano.gns.cri.nz,user=volcano/home/volcano/data/omi/archive'

#folder for visualizations
vizdir = '/home/sherburn/Work/GeoNet/omi/viz'

#for each day, look 'back' at nprevdays of data
nprevdays = 2

#each time script is run, fully process nprocdays back from current day
nprocdays = 3

In [ ]:
def prevdays(date, ndays):
    #find previous ndays of dates given a date string
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    days = []
    for d in range(ndays):
        calcd = (dt+ timedelta(days=-d)).date()
        days.append(calcd.strftime('%Y-%m-%d'))
    return days

## Loops and visualize

In [ ]:
current_utcday = datetime.datetime.utcnow().strftime('%Y-%m-%d')
print ('current utc day = ', current_utcday)
print ()

processdays = prevdays(current_utcday, nprocdays)

for procday in processdays:
    print ('process day: ', procday)
    searchdays = prevdays(procday, nprevdays)
    files = findfiles(searchdays)
    for volcano in volcanoes:
        name =  volcano['name']          
        #find expected utc satellite pass time for day and volcano
        passutc = passtime(name, procday)
        print ('for volcano = {}, passtime = {}'.format(name, passutc))
    
        nearest = nearest_file(files, passutc)
        visfile = nearest[0]
        print ('nearest file is ', visfile)
        #attempt to visualize the nearest file
        visualize(procday, visfile, volcanoes, name, vizdir)
    print ()